리뷰에서 계절감, 시간대 추천하기
배열할 때 row에 추천 계절, 추천 시간대 (아침, 오후, 저녁)
전이학습모델 -> 리뷰데이터 함축, 문서수준에서 아침, 오후, 저녁 단어와 유사도가 깊은 단어들 고르기 -> 3개 중 그 정도가 가장 큰 것 아이콘 표현

In [14]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader

# 아침, 점심, 저녁 데이터셋 생성
morning_data = ["아침 출근길에 라디오를 들었다. 보통은 노래를 들으면서 가는 편이다. 아침형 인간이 아니라 이른 아침에 움직이려면 어떤 식으로든 에너지를 끌어올리는 시간이 필요하기 때문이다. 첫 번째는 커피, 두 번째는 음악으로 도움을 받는다. 스스로 해결하려면 점심깨나 되어야 가능할 일이다. 느릿하게 서서히 오르는 걸. 그러고 나면 강의 하나는 끝나버릴 시간이 되어버리니까 기다려줄 수가 없다. 오전 10시 강의가 있는 날에는 무척 분주하다. 요즘은 예전보다 아침잠이 줄어서 7시쯤이면 눈이 떠지니 좀 나아졌다. 나는 눈 뜨자마자 핸드폰을 연다. 메일 확인 후, 세수를 하고 로션을 바른다. 그리고 차가운 커피를 마시면서, 간단하게 먹을 간식을 준비해 놓거나 빨래를 확인하고, 옷을 입는다. 식탁에 앉아 커피를 마실 시간이 없다. 커피 한 잔, 집안일 한 번이 반복되다가 커피를 다 마시면 양치질을 하고 집을 나선다.그날 공교롭게도 라디오에서 나온 사연이 '초라하지 않은 아침을 보내고 싶어요.'였다. 앞에 이야기는 흘려들어서 아무 생각도 나질 않는다. '초라'라는 단어에 정신이 번뜩했다. 나도 혹시 초라한 아침을 보내고 있지는 않은가, 하는 생각이 들었기 때문이다. 내 아침은 어떤 단어로 표현될까. 대충 떠올려봐도 긍정적인 단어와 거리가 멀다. 정신없고 부산스러운 아침이었다. 커피는 여유로운 느낌인데 나와 만나면 바쁨을 표현하는 단어가 되어 버렸다. 잠시나마 앉아서 커피 한 잔도 못 마시는 여자의 출근준비가 제목으로 어울린달까. 시간이 있을 때 아무 것도 하지 않으면 어떠냐는 질문을 받아 보았는데 나는 견디지 못하겠다는 대답을 했다. 어떤 사람이 내게 말했다. 예쁜 걸 좋아하고 그런 것만 찾아서 하면 다른 사람들이 예쁜 걸 보면 나를 떠올린다는 말이었다. 주변 사람들은 나를 보면 바쁨을 먼저 생각한다. 이 글을 쓰는 지금도 아침 커피를 마시며 출근 준비를 하는 중이다. 한 가지만 하면 시간이 아까워서, 커피 마시면서 서류를 뽑다가 글도 쓴다. 나의 아침은 바쁘다. 오늘도, 내일도 똑같이 바쁠 것이다. 시간 여유는 없지만, 있다 해도 내가 못 견딜 일이다. 그러니까 내 아침은 초라하지 않다. 내 아침 풍경은 매일 무언가 하려고 애쓰는 모습이다. 이제 커피잔이 비었으니 일하러 가야겠다."]
lunch_data = ["점심은 샐러드를 먹었다.", "점심에는 떡볶이를 먹었다.", "점심은 도시락을 가져왔다."]
dinner_data = ["저녁에는 스파게티를 먹었다.", "저녁에는 치킨을 주문했다.", "저녁에는 김치찌개를 끓였다."]

# 모든 데이터를 하나의 리스트로 통합
all_data = morning_data + lunch_data + dinner_data

# 레이블 지정
labels = [0] * len(morning_data) + [1] * len(lunch_data) + [2] * len(dinner_data)

# BERT 토크나이저 및 모델 불러오기
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 훈련 데이터셋 생성
class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {"text": self.texts[idx], "label": self.labels[idx]}

train_dataset = CustomDataset(all_data, labels)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Optimizer 및 손실 함수 설정
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# Fine-tuning
model.train()
epochs = 3

for epoch in range(epochs):
    for batch in train_dataloader:
        inputs = tokenizer(batch["text"], return_tensors="pt", padding=True, truncation=True)
        labels = torch.tensor(batch["label"])

        optimizer.zero_grad()
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Fine-tuned 모델 저장
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

c:\Users\Playdata\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Playdata\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\Playdata\AppData\Local\Temp\ipykernel_22684\677492982.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch["label"])


('./finetuned_model\\tokenizer_config.json',
 './finetuned_model\\special_tokens_map.json',
 './finetuned_model\\vocab.txt',
 './finetuned_model\\added_tokens.json')

In [17]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Fine-tuned 모델 및 토크나이저 불러오기
model_directory = "./finetuned_model"  # Fine-tuning 시 생성된 모델 디렉토리의 경로
tokenizer = BertTokenizer.from_pretrained(model_directory)
model = BertForSequenceClassification.from_pretrained(model_directory)

def predict_class(input_text):
    # 텍스트를 BERT 입력 형식으로 변환
    inputs = tokenizer(input_text, return_tensors="pt")

    # 모델을 통해 예측 수행
    with torch.no_grad():
        outputs = model(**inputs)

    # 예측 결과 출력
    probs = torch.nn.functional.softmax(outputs.logits, dim=1).squeeze().tolist()
    predicted_class = torch.argmax(outputs.logits).item()

    class_labels = ['아침', '점심', '저녁']

    return {
        "predicted_class": class_labels[predicted_class],
        "class_probabilities": probs
    }

# 예측 테스트
input_text = "어둡다"
result = predict_class(input_text)
print("Input Text:", input_text)
print("Predicted Class:", result["predicted_class"])
print("Class Probabilities:", result["class_probabilities"])


Input Text: 어둡다
Predicted Class: 아침
Class Probabilities: [0.35767662525177, 0.2996659278869629, 0.3426574468612671]


In [11]:
import torch
from transformers import BertModel, BertTokenizer
from scipy.spatial.distance import cosine

# Load KoBERT model and tokenizer
model_name = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Define the target words
target_words = ['아침', '점심', '저녁']

def get_word_embedding(word):
    # Tokenize the word and obtain its embeddings
    tokens = tokenizer(word, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

def calculate_cosine_similarity(embedding1, embedding2):
    # Calculate cosine similarity between two embeddings
    return 1 - cosine(embedding1, embedding2)
def find_most_similar_word(text):
    # Tokenize the input text and obtain its embeddings
    tokens = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**tokens)
    input_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

    # Calculate cosine similarity with each target word
    similarities = [calculate_cosine_similarity(input_embedding, get_word_embedding(target_word)) for target_word in target_words]

    # Find the most similar word
    most_similar_index = similarities.index(max(similarities))
    most_similar_word = target_words[most_similar_index]

    return most_similar_word, similarities

# Example usage
input_text = "오늘 점심에 뭐 먹었어?"
most_similar_word, similarity_scores = find_most_similar_word(input_text)
print(f"The most similar word to the input text is '{most_similar_word}' with similarity scores: {similarity_scores}")


The most similar word to the input text is '아침' with similarity scores: [0.4304794371128082, 0.4304794371128082, 0.4304794371128082]


In [ ]:
#계절 추천
#'봄'과 관련 깊은 단어들 빈도 ...  4개 비교해서 가장 유사도 높은
